In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv(r"C:\Users\Hp\Desktop\MyWorks\Innomatics\Machine Learning\Assessments\KNN -2\diamonds.csv")
data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [3]:
x = data.drop('price',axis=1)
y = data['price']
x.shape, y.shape

((53940, 9), (53940,))

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
x_test, x_train, y_test, y_train = train_test_split(x,y,test_size = 0.25, random_state = 42)
x_test.shape, x_train.shape, y_test.shape, y_train.shape

((40455, 9), (13485, 9), (40455,), (13485,))

In [6]:
from sklearn.preprocessing import RobustScaler,OrdinalEncoder

In [7]:
from sklearn.compose import ColumnTransformer

In [8]:
cut = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']

color = ['J','I','H','G','F','E','D']

clarity = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']


In [9]:
rs = RobustScaler()
oe = OrdinalEncoder(categories=[cut,color,clarity])
rs

RobustScaler()

In [10]:
ct = ColumnTransformer([
    ('rs',rs,['carat','depth','table','x','y','z']),
    ('oe',oe,['cut','color','clarity'])],
    verbose_feature_names_out = False,
    remainder = 'passthrough').set_output(transform='pandas')
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('rs', RobustScaler(),
                                 ['carat', 'depth', 'table', 'x', 'y', 'z']),
                                ('oe',
                                 OrdinalEncoder(categories=[['Fair', 'Good',
                                                             'Very Good',
                                                             'Premium',
                                                             'Ideal'],
                                                            ['J', 'I', 'H', 'G',
                                                             'F', 'E', 'D'],
                                                            ['I1', 'SI2', 'SI1',
                                                             'VS2', 'VS1',
                                                             'VVS2', 'VVS1',
                                                             'IF']]),
                                 ['cut', 'color', 'clarity'])],
                  verbose_feature_names_out=False)

In [11]:
x_train_transform = ct.fit_transform(x_train)
x_train_transform .head()

,carat,depth,table,x,y,z,cut,color,clarity
1388,-0.718750,0.142857,-0.333333,-0.945355,-0.939560,-0.929204,4.0,3.0,6.0
50052,-0.187500,-1.357143,0.000000,-0.142077,-0.159341,-0.230088,2.0,4.0,5.0
41645,-0.468750,0.142857,-0.666667,-0.513661,-0.532967,-0.504425,4.0,5.0,5.0
42377,-0.421875,-0.785714,0.000000,-0.426230,-0.450549,-0.477876,3.0,5.0,5.0
17244,1.328125,0.285714,-0.666667,0.950820,0.912088,0.964602,4.0,5.0,1.0


In [12]:
x_test_transform = ct.transform(x_test)
x_test_transform.head()

,carat,depth,table,x,y,z,cut,color,clarity
35965,-0.703125,2.142857,0.333333,-0.956284,-0.956044,-0.840708,1.0,5.0,5.0
52281,0.218750,-0.071429,-0.333333,0.185792,0.197802,0.194690,4.0,0.0,2.0
6957,0.546875,-0.571429,0.333333,0.469945,0.439560,0.415929,3.0,0.0,3.0
9163,0.500000,-0.857143,-0.333333,0.453552,0.434066,0.380531,4.0,4.0,1.0
50598,-0.140625,-0.071429,0.000000,-0.147541,-0.131868,-0.132743,4.0,4.0,4.0


In [13]:
from sklearn.metrics.pairwise import euclidean_distances

In [14]:
query=x_test_transform.sample(n=1,random_state=42)
query

,carat,depth,table,x,y,z,cut,color,clarity
21578,1.265625,-1.0,0.666667,0.95082,0.934066,0.858407,3.0,1.0,3.0


In [15]:
dist = pd.DataFrame(euclidean_distances(x_train_transform,query)).sort_values(by=0,ascending=True)
dist.head()

,0
10457,0.256289
12325,0.443419
4994,0.588602
6975,0.609383
12677,0.650491


In [16]:
top_5 = dist.iloc[:5,:]
top_5

,0
10457,0.256289
12325,0.443419
4994,0.588602
6975,0.609383
12677,0.650491


In [23]:
from sklearn.neighbors import KNeighborsRegressor

In [24]:
knnr=KNeighborsRegressor(n_neighbors=5)

In [25]:
knnr.fit(x_train_transform,y_train)

KNeighborsRegressor()

In [26]:
knnr.predict(query)

array([7552.])

In [27]:
knnr.score(x_train_transform,y_train)

0.9641754246795415

In [28]:
y_pred=knnr.predict(x_test_transform)

In [29]:
from sklearn.metrics import r2_score

In [30]:
r2_score(y_test,y_pred)

0.9446047734904226